<h2><center>IIC2440 – Procesamiento de Datos Masivos</center></h2>
<h3><center>Tarea 1: Análisis sesiones parlamentarias</center></h3>


**Integrantes:**
- Franco Chiappe
- Tomás Pérez

### Librerias

In [1]:
# Importar librerias 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import unicodedata

## Parte 1 [1 pto] - Procesando los datos

In [2]:
# Cargar los datos
parlamentarios_info = pd.read_csv('participacion/parlamentarios_info_general.csv')
descripcion_2023_1 = pd.read_csv('participacion/participacion_descripcion_2023_1.csv')
descripcion_2023_2 = pd.read_csv('participacion/participacion_descripcion_2023_2.csv')
descripcion_2024 = pd.read_csv('participacion/participacion_descripcion_2024.csv')
df = pd.concat([descripcion_2023_1, descripcion_2023_2, descripcion_2024], ignore_index=True)

### Pre-Procesamiento de Datos

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68161 entries, 0 to 68160
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_PARTICIPACION    68161 non-null  int64  
 1   PARLAMENTARIO_ID    68161 non-null  int64  
 2   FECHA               68161 non-null  object 
 3   LEGISLATURA         68161 non-null  int64  
 4   SESION              68161 non-null  int64  
 5   TIPO_SESION         68161 non-null  object 
 6   TIPO_PARTICIPACION  68161 non-null  object 
 7   CAMARA              68161 non-null  object 
 8   PARLAMENTARIOS      68161 non-null  object 
 9   TEXTO_PRINCIPAL     68161 non-null  object 
 10  TEXTO_ANTECEDENTE   32393 non-null  object 
 11  DESCRIPCION_DEBATE  68161 non-null  object 
 12  BOLETIN_ID          20593 non-null  float64
dtypes: float64(1), int64(4), object(8)
memory usage: 6.8+ MB


Nos damos cuenta que en las columnas:

- `TEXTO_ANTECEDENTE` tiene 35768  datos nulos
- `BOLETIN_ID` tiene 47568 datos nulos

Decidimos eliminar la columna `TEXTO_ANTECEDENTE`, porque presentaba un alto porcentaje de datos nulos y no aportaba informacion relevante. Por otro parte, no eliminamos la columna `BOLETIN_ID`, esta columna podria resultar util para analisis futuros.

In [4]:
df = df.drop(columns=["TEXTO_ANTECEDENTE"])

### Limpieza de texto

In [5]:
def limpiar_texto(texto):
        
    texto = texto.lower()
    texto = texto.strip()
    
    texto = re.sub(r'[^\w\s]', '', texto)
    texto = re.sub(r'\s+', ' ', texto)

    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')

    return texto

Se usa la funcion ***limpiar_texto*** el objetivo es normalizar los textos para facilitar la extraccion de keywords. Esta funcion convierte todo el contenido a minuscula, elimina tildes, signos de puntuación y espacios innecesarios, dejando un text limpio. Esto permite detectar con mayor precision los temas.

In [6]:
# Aplicar la funcion de limpieza a las columnas
df['DESCRIPCION_DEBATE'] = df['DESCRIPCION_DEBATE'].apply(limpiar_texto)
df['TEXTO_PRINCIPAL'] = df['TEXTO_PRINCIPAL'].apply(limpiar_texto)

# Juntamos las columnas
df["TEXTO_COMBINADO"] = df["TEXTO_PRINCIPAL"] + " " + df["DESCRIPCION_DEBATE"]
df["TEXTO_COMBINADO"].head()

0    la senora allende ngracias presidente npor su ...
1    la senora allende ngracias presidente nen prim...
2    proyecto de acuerdo de los honorables senadore...
3    de la senora allende n a la municipalidad de v...
4    proyecto de acuerdo de los honorables senadore...
Name: TEXTO_COMBINADO, dtype: object

### Temas y palabras claves

In [7]:
temas = {
    "salud": ["salud", "hospital", "hospitales" "medico", "vacuna", "enfermedad", "medicina", "medicamento"],
    "educacion": ["educacion", "colegio", "escuela", "universidad", "profesor", "alumno", "estudiante", "educador", "colegios"],
    "delincuencia": ["delincuencia", "carabinero", "robo", "narcotrafico", "seguridad", "delito", "delincuente", "delincuentes"],
    "medio ambiente": ["medioambiente", "naturaleza", "contaminacion", "reciclaje", "agua", "bosque", "bosques", "flora", "fauna", "ecosistema", "ecosistemas"],
    "economia": ["economia", "dinero", "impuesto", "trabajo", "impuestos", "empresa", "empresas", "negocio", "negocios", "inversion", "inversiones", "financiero", "financieros"],
    "constitucion": ["constitucion", "constitucional"],
    "vivienda": ["vivienda", "casa", "alquiler", "arrendar"],
    "transporte": ["transporte", "metro", "camino", "micro"],
    "derechos humanos": ["derechos humanos", "derechos", "humanos", "discriminacion", "discriminacion", "discriminados", "discriminados", "discriminacion", "discriminaciones", "humanitaria"],
    "legislacion": ["legislacion", "ley", "leyes", "legislativo", "legislativa"],
    "justicia": ["justicia", "justicia", "jueces", "juez", "tribunal", "tribunales"],
}

In [8]:
def asignar_keywords(texto):
    palabras = texto.split()
    etiquetas = set()
    for tema, claves in temas.items():
        for palabra in claves:
            if (palabra in palabras):
                etiquetas.add(tema)
       
    return list(etiquetas)

In [9]:
# Se agrupan por descripción debate, para poder aplicar la función de asignar_keywords a cada grupo de manera más eficiente
grouped_df = df.groupby("DESCRIPCION_DEBATE").first().reset_index()
grouped_df["KEYWORDS"] = grouped_df["TEXTO_COMBINADO"].apply(asignar_keywords)

df = df.merge(grouped_df[["TEXTO_COMBINADO", "KEYWORDS"]], on="TEXTO_COMBINADO", how="left")
df["KEYWORDS"].value_counts()

KEYWORDS
[]                                                                          4836
[legislacion]                                                               4060
[economia]                                                                   814
[salud]                                                                      672
[economia, legislacion]                                                      650
                                                                            ... 
[salud, delincuencia, justicia, legislacion]                                   1
[economia, medio ambiente, justicia, derechos humanos, salud]                  1
[economia, legislacion, educacion, justicia, transporte]                       1
[constitucion, economia, legislacion, derechos humanos, vivienda, salud]       1
[transporte, medio ambiente, justicia, derechos humanos]                       1
Name: count, Length: 524, dtype: int64

### Conclusion:
Se realizo el preprocesamiento de las intervenciones parlamentarias con el objetivo de identificar las tematicas. Para esto, se aplico una funcion de limpieza que normalizo el texto (conversion a minusculas, eliminacion de tildes, puntuacion y espacios innecesarios). Luego, se unieron las columnas `TEXTO_ANTECEDENTE` y `DESCRIPCION_DEBATE` para generar un texto combinado. Finalmente, se aplico una funcion que detecta las palabras clave y asigna a cada intervencion una o mas temas, permitiendo categorizar los discursos parlamentarios segun temas como salud, educación, delincuencia, entre otros. Para optimizar la asignación, se agruparon las filas por la columna `DESCRIPCION_DEBATE`, ya que muchas compartian o eran de un mismo debate, para así acortar el tiempo de ejecución de asignación

## Parte 2 - [1 pts] - Modelamiento

### Revisión Tipo de Datos

In [10]:
parlamentarios_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1199 entries, 0 to 1198
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PARLAMENTARIO_ID       1199 non-null   int64  
 1   NOMBRE_COMPLETO        1199 non-null   object 
 2   VIGENTE                1199 non-null   int64  
 3   FECHA_NACIMIENTO       206 non-null    object 
 4   REDES_SOCIALES         0 non-null      float64
 5   SITIOS_WEB             0 non-null      float64
 6   CAMARA_ACTUAL          206 non-null    object 
 7   CAMARA                 206 non-null    object 
 8   INICIO                 206 non-null    float64
 9   FINAL                  206 non-null    float64
 10  PARTIDO_POLITICO       155 non-null    object 
 11  UNIDAD_QUE_REPRESENTA  205 non-null    object 
dtypes: float64(4), int64(2), object(6)
memory usage: 112.5+ KB


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68161 entries, 0 to 68160
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_PARTICIPACION    68161 non-null  int64  
 1   PARLAMENTARIO_ID    68161 non-null  int64  
 2   FECHA               68161 non-null  object 
 3   LEGISLATURA         68161 non-null  int64  
 4   SESION              68161 non-null  int64  
 5   TIPO_SESION         68161 non-null  object 
 6   TIPO_PARTICIPACION  68161 non-null  object 
 7   CAMARA              68161 non-null  object 
 8   PARLAMENTARIOS      68161 non-null  object 
 9   TEXTO_PRINCIPAL     68161 non-null  object 
 10  DESCRIPCION_DEBATE  68161 non-null  object 
 11  BOLETIN_ID          20593 non-null  float64
 12  TEXTO_COMBINADO     68161 non-null  object 
 13  KEYWORDS            19803 non-null  object 
dtypes: float64(1), int64(4), object(9)
memory usage: 7.3+ MB


Falta realizar el trabajo de las tablas y las consultas que se piden:

Luego, ten presente que harás las siguientes consultas:


¿Cuáles son las top 5 temáticas más tratadas, para cada mes en el que hay registros?

Media móvil de intervenciones por partido político con un intervalo de 3 meses.

Para cada trimestre, ver el tema principal tratado por cada partido político.

Poder pararse en un mes y ver cuál es el top 3 de temáticas tratadas por cada partido. Esto vas a querer repetirlo para algunos meses.


## Parte 3 - [1.5 pts] - BigQuery

## Parte 4 - [2 pts] - Informe>

## Parte 5 - [0.5 pts] - Resumen ejecutivo